In [146]:
import os

In [147]:
%pwd

'c:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [148]:
#os.chdir("./data-science-project/")
%pwd

'c:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [149]:
import pandas as pd

In [150]:
df = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [151]:
df.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [152]:
df.shape

(1599, 12)

In [153]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict


In [154]:
from src.DataScienceProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from src.DataScienceProject.utils.common import read_yaml, create_directories

In [155]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            unzip_data_dir=config.unzip_data_dir,
            STATUS_FILE=config.STATUS_FILE,
            all_schema=schema,
        )
        return data_validation_config

In [156]:
from src.DataScienceProject import logger

In [157]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config
        logger.info(f"Data Validation config: {self.config}")
    def validate_cols_name(self) -> bool:
        try:
            logger.info("Start data validation")
            validation_status = True

            df = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(df.columns)

            all_schema_cols = self.config.all_schema.keys()
            for col in all_cols:
                if col not in all_schema_cols:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation Name Status: {validation_status}\n")
                    logger.info(f"Column {col} esta faltando")
                    break
                
                    
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation Name Status: {validation_status}\n")

            validation_col_name_status = validation_status
            return validation_col_name_status
        
        except Exception as e:
            logger.error(f"Erro no data validation: {e}")
            raise e
    
    def validate_type_cols(self) -> bool:
        try:
            status = True
            df = pd.read_csv(self.config.unzip_data_dir)
            all_schema_cols = self.config.all_schema.keys()
            
            for col in all_schema_cols:
                if col not in df.columns:
                    logger.warning(f"Column  not found in dataframe")
                    return False
                expected_type = self.config.all_schema[col]
                actual_type = df[col].dtype
                if expected_type != actual_type:
                    status = False
                    with open(self.config.STATUS_FILE, 'a') as f:
                        f.write(f"Validation type Status : {status} \n")
                    logger.info(f"Column {col} has wrong type. Expected: {expected_type}, Actual: {actual_type}")
                    break
            
            with open(self.config.STATUS_FILE, 'a') as f:
                f.write(f"Validation type Status: {status} \n")
             
            logger.info("Type validation passed")
            status_type_validation = status
            return status_type_validation
        except Exception as e:
            logger.error(f"Erro no type validation: {e}")
            raise e

In [158]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()

    data_validation = DataValidation(config=data_validation_config)
    status_name = data_validation.validate_cols_name()
    status_type = data_validation.validate_type_cols()
    logger.info(f"Data validation name status: {status_name}")
    logger.info(f"Data validation type status: {status_type}")
except Exception as e:
    logger.error(f"Erro no data validation: {e}")
    raise e

[2026-02-04 21:06:13,828: INFO: common: YAML file: config\config.yaml carregado com sucesso:]
[2026-02-04 21:06:13,830: INFO: common: YAML file: params.yaml carregado com sucesso:]
[2026-02-04 21:06:13,835: INFO: common: YAML file: schema.yaml carregado com sucesso:]
[2026-02-04 21:06:13,835: INFO: common: Diretório criado em: artifacts:]
[2026-02-04 21:06:13,840: INFO: common: Diretório criado em: artifacts/data_validation:]
[2026-02-04 21:06:13,841: INFO: 257978153: Data Validation config: DataValidationConfig(root_dir='artifacts/data_validation', unzip_data_dir='artifacts/data_ingestion/winequality-red.csv', STATUS_FILE='artifacts/data_validation/status.txt', all_schema=ConfigBox({'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int